# I. Modélisation statistique

In [25]:
!pip install s3fs
!pip install pandas
!pip install scikit-learn==1.2.2
!pip install imbalanced-learn
!pip install matplotlib

^C


## I.1. Chargement et préparation des données

### A) Importation des données

In [ ]:
import pandas as pd

# Importation des données avec code_insee en index
donnees = pd.read_csv("bdd_finale.csv", sep=',', encoding='utf-8', index_col=0)
donnees.head()

### B) Formatage des données

Nous allons pouvoir observer le type des données que nous avons. Cela va nous permettre de déterminer si on a besoin de modifier certaines variables pour qu'elles soient bien en variables catégorielles. 

In [ ]:
donnees.dtypes

In [ ]:
# Conversion des variables en variables catégorielles
donnees["beneficiaire_trans_eco"] = donnees["beneficiaire_trans_eco"].astype('category')
donnees["ecoquartiers"] = donnees["ecoquartiers"].astype('category')
donnees["beneficiaire_prog"] = donnees["beneficiaire_prog"].astype('category')
donnees["gridens7"] = donnees["gridens7"].astype('category')
donnees["departement"] = donnees["departement"].astype('category')
donnees["gare_tgv"] = donnees["gare_tgv"].astype('category')
donnees["CSP_maire"] = donnees["CSP_maire"].astype('category')


# Cas de la variable climat 
donnees = pd.get_dummies(donnees, columns = ["climat"]) # On crée des variables binaires pour la variable climat car on a des chaînes de caractères

In [ ]:
donnees.dtypes

Nous allons observer si on a des données manquantes dans notre jeu de données.

In [ ]:
donnees.isna().sum()

### C) Séparation de la variable à expliquer et des variables explicatives

In [ ]:
# Définir X et y pour avoir les variables explicatives et la variable à expliquer beneficiaire_trans_eco
X = donnees.drop(columns=['beneficiaire_trans_eco'])

# Variable à expliquer : beneficiaire_trans_eco
y = donnees["beneficiaire_trans_eco"]

In [ ]:
# import numpy as np

# # Convertir les listes en tableaux numpy
# X = np.array(X)
# y = np.array(y)

In [ ]:
print("Dimensions de X:", X.shape)
print("Dimensions de y:", y.shape)

In [ ]:
# import numpy as np

# np.unique(y)

## I.2. Modélisation

Nous allons d'abord diviser notre jeu de données en jeux d'apprentissage et de test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

## I.2.1. Régression logistique

### A) Construction des modèles

Nous allons maintenant définir notre modèle de régression logistique, sans pénalité.

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE

etapes_reg_log_oversampler = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('oversampler', RandomOverSampler()), # Sur-échantillonnage
        ('reg_log', LogisticRegression()), # Régression logistique
        ]

etapes_reg_log_smote = [('std_scaler', StandardScaler()), # Standardisation des variables pour ne pas donner plus d'importance à une variable qu'à une autre
        ('smote', SMOTE()), # Sur-échantillonnage
        ('reg_log', LogisticRegression()), # Régression logistique
        ]

etapes_reg_log_adasyn = [('std_scaler', StandardScaler()), # Standardisation des variables pour ne pas donner plus d'importance à une variable qu'à une autre
        ('adasyn', ADASYN()), # Sur-échantillonnage
        ('reg_log', LogisticRegression()), # Régression logistique
        ]

etapes_reg_log_bordeline_smote = [('std_scaler', StandardScaler()), # Standardisation des variables pour ne pas donner plus d'importance à une variable qu'à une autre
        ('bordeline_smote', BorderlineSMOTE()), # Sur-échantillonnage
        ('reg_log', LogisticRegression()), # Régression logistique
        ]

# Création des pipelines
modele_reg_log_smote  = Pipeline(steps=etapes_reg_log_smote) 
modele_reg_log_oversampler  = Pipeline(steps=etapes_reg_log_oversampler) 
modele_reg_log_adasyn  = Pipeline(steps=etapes_reg_log_adasyn)
modele_reg_log_bordeline_smote  = Pipeline(steps=etapes_reg_log_bordeline_smote)

On entraîne le modèle sur les données pour estimer les coefficients. 

In [ ]:
modele_reg_log_smote.fit(X_train, y_train)

In [ ]:
modele_reg_log_oversampler.fit(X_train, y_train)

In [ ]:
modele_reg_log_adasyn.fit(X_train, y_train)

In [ ]:
modele_reg_log_bordeline_smote.fit(X_train, y_train)

On cherche à prédire de nouvelles données grâce aux données test

In [ ]:
y_proba_reg_log_smote = modele_reg_log_smote.predict_proba(X_test)[:,1]
y_pred_reg_log_smote = modele_reg_log_smote.predict(X_test)

In [ ]:
y_proba_reg_log_oversampler = modele_reg_log_oversampler.predict_proba(X_test)[:, 1]
y_pred_reg_log_oversampler = modele_reg_log_oversampler.predict(X_test)

In [ ]:
y_proba_reg_log_adasyn = modele_reg_log_adasyn.predict_proba(X_test)[:, 1]
y_pred_reg_log_adasyn = modele_reg_log_adasyn.predict(X_test)

In [ ]:
y_proba_reg_log_bordeline_smote = modele_reg_log_bordeline_smote.predict_proba(X_test)[:, 1]
y_pred_reg_log_bordeline_smote = modele_reg_log_bordeline_smote.predict(X_test)

#### B) Mesure de la performance des modèles initiaux

Nous allons maintenant évaluer la performance de notre modèle

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, log_loss
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, cohen_kappa_score


# Calcul des métriques
auc_roc_reg_log_smote = roc_auc_score(y_test, y_proba_reg_log_smote)
auc_pr_reg_log_smote  = average_precision_score(y_test, y_proba_reg_log_smote)
log_loss_value_reg_log_smote  = log_loss(y_test, y_proba_reg_log_smote)
precision_reg_log_smote  = precision_score(y_test, y_pred_reg_log_smote)
recall_reg_log_smote  = recall_score(y_test, y_pred_reg_log_smote)
f1_reg_log_smote = f1_score(y_test, y_pred_reg_log_smote)
mcc_reg_log_smote  = matthews_corrcoef(y_test, y_pred_reg_log_smote)
balanced_acc_reg_log_smote  = balanced_accuracy_score(y_test, y_pred_reg_log_smote)
specificity_reg_log_smote  = confusion_matrix(y_test, y_pred_reg_log_smote)[0, 0] / (confusion_matrix(y_test, y_pred_reg_log_smote)[0, 0] + confusion_matrix(y_test, y_pred_reg_log_smote)[0, 1])
cohen_kappa_reg_log_smote  = cohen_kappa_score(y_test, y_pred_reg_log_smote)

# Affichage des résultats
print(f"AUC-ROC: {auc_roc_reg_log_smote }")
print(f"AUC-PR: {auc_pr_reg_log_smote }")
print(f"Log Loss: {log_loss_value_reg_log_smote }")
print(f"Precision: {precision_reg_log_smote }")
print(f"Recall: {recall_reg_log_smote }")
print(f"F1 Score: {f1_reg_log_smote }")
print(f"MCC: {mcc_reg_log_smote }")
print(f"Balanced Accuracy: {balanced_acc_reg_log_smote }")
print(f"Specificity: {specificity_reg_log_smote }")
print(f"Cohen's Kappa: {cohen_kappa_reg_log_smote }")

In [ ]:
# Calcul des métriques

# Calcul des métriques
auc_roc_reg_log_oversampler = roc_auc_score(y_test, y_proba_reg_log_oversampler)
auc_pr_reg_log_oversampler = average_precision_score(y_test, y_proba_reg_log_oversampler)
log_loss_value_reg_log_oversampler = log_loss(y_test, y_proba_reg_log_oversampler)
precision_reg_log_oversampler = precision_score(y_test, y_pred_reg_log_oversampler)
recall_reg_log_oversampler = recall_score(y_test, y_pred_reg_log_oversampler)
f1_reg_log_oversampler = f1_score(y_test, y_pred_reg_log_oversampler)
mcc_reg_log_oversampler = matthews_corrcoef(y_test, y_pred_reg_log_oversampler)
balanced_acc_reg_log_oversampler = balanced_accuracy_score(y_test, y_pred_reg_log_oversampler)
specificity_reg_log_oversampler = confusion_matrix(y_test, y_pred_reg_log_oversampler)[0, 0] / (confusion_matrix(y_test, y_pred_reg_log_oversampler)[0, 0] + confusion_matrix(y_test, y_pred_reg_log_oversampler)[0, 1])
cohen_kappa_reg_log_oversampler = cohen_kappa_score(y_test, y_pred_reg_log_oversampler)

# Affichage des résultats
print(f"AUC-ROC: {auc_roc_reg_log_oversampler}")
print(f"AUC-PR: {auc_pr_reg_log_oversampler}")
print(f"Log Loss: {log_loss_value_reg_log_oversampler}")
print(f"Precision: {precision_reg_log_oversampler}")
print(f"Recall: {recall_reg_log_oversampler}")
print(f"F1 Score: {f1_reg_log_oversampler}")
print(f"MCC: {mcc_reg_log_oversampler}")
print(f"Balanced Accuracy: {balanced_acc_reg_log_oversampler}")
print(f"Specificity: {specificity_reg_log_oversampler}")
print(f"Cohen's Kappa: {cohen_kappa_reg_log_oversampler}")

In [ ]:
# Calcul des métriques

# Calcul des métriques
auc_roc_reg_log_adasyn = roc_auc_score(y_test, y_proba_reg_log_adasyn)
auc_pr_reg_log_adasyn = average_precision_score(y_test, y_proba_reg_log_adasyn)
log_loss_value_reg_log_adasyn = log_loss(y_test, y_proba_reg_log_adasyn)
precision_reg_log_adasyn = precision_score(y_test, y_pred_reg_log_adasyn)
recall_reg_log_adasyn = recall_score(y_test, y_pred_reg_log_adasyn)
f1_reg_log_adasyn = f1_score(y_test, y_pred_reg_log_adasyn)
mcc_reg_log_adasyn = matthews_corrcoef(y_test, y_pred_reg_log_adasyn)
balanced_acc_reg_log_adasyn = balanced_accuracy_score(y_test, y_pred_reg_log_adasyn)
specificity_reg_log_adasyn = confusion_matrix(y_test, y_pred_reg_log_adasyn)[0, 0] / (confusion_matrix(y_test, y_pred_reg_log_adasyn)[0, 0] + confusion_matrix(y_test, y_pred_reg_log_adasyn)[0, 1])
cohen_kappa_reg_log_adasyn = cohen_kappa_score(y_test, y_pred_reg_log_adasyn)

# Affichage des résultats
print(f"AUC-ROC: {auc_roc_reg_log_adasyn}")
print(f"AUC-PR: {auc_pr_reg_log_adasyn}")
print(f"Log Loss: {log_loss_value_reg_log_adasyn}")
print(f"Precision: {precision_reg_log_adasyn}")
print(f"Recall: {recall_reg_log_adasyn}")
print(f"F1 Score: {f1_reg_log_adasyn}")
print(f"MCC: {mcc_reg_log_adasyn}")
print(f"Balanced Accuracy: {balanced_acc_reg_log_adasyn}")
print(f"Specificity: {specificity_reg_log_adasyn}")
print(f"Cohen's Kappa: {cohen_kappa_reg_log_adasyn}")

In [ ]:
# Calcul des métriques

auc_roc_reg_log_borderline_smote = roc_auc_score(y_test, y_proba_reg_log_bordeline_smote)
auc_pr_reg_log_borderline_smote = average_precision_score(y_test, y_proba_reg_log_bordeline_smote)
log_loss_value_reg_log_borderline_smote = log_loss(y_test, y_proba_reg_log_bordeline_smote)
precision_reg_log_borderline_smote = precision_score(y_test, y_pred_reg_log_bordeline_smote)
recall_reg_log_borderline_smote = recall_score(y_test, y_pred_reg_log_bordeline_smote)
f1_reg_log_borderline_smote = f1_score(y_test, y_pred_reg_log_bordeline_smote)
mcc_reg_log_borderline_smote = matthews_corrcoef(y_test, y_pred_reg_log_bordeline_smote)
balanced_acc_reg_log_borderline_smote = balanced_accuracy_score(y_test, y_pred_reg_log_bordeline_smote)
specificity_reg_log_borderline_smote = confusion_matrix(y_test, y_pred_reg_log_bordeline_smote)[0, 0] / (confusion_matrix(y_test, y_pred_reg_log_bordeline_smote)[0, 0] + confusion_matrix(y_test, y_pred_reg_log_bordeline_smote)[0, 1])
cohen_kappa_reg_log_borderline_smote = cohen_kappa_score(y_test, y_pred_reg_log_bordeline_smote)

# Affichage des résultats
print(f"AUC-ROC: {auc_roc_reg_log_borderline_smote}")
print(f"AUC-PR: {auc_pr_reg_log_borderline_smote}")
print(f"Log Loss: {log_loss_value_reg_log_borderline_smote}")
print(f"Precision: {precision_reg_log_borderline_smote}")
print(f"Recall: {recall_reg_log_borderline_smote}")
print(f"F1 Score: {f1_reg_log_borderline_smote}")
print(f"MCC: {mcc_reg_log_borderline_smote}")
print(f"Balanced Accuracy: {balanced_acc_reg_log_borderline_smote}")
print(f"Specificity: {specificity_reg_log_borderline_smote}")
print(f"Cohen's Kappa: {cohen_kappa_reg_log_borderline_smote}")


In [ ]:
# Comparaison des courbes ROC

from sklearn.metrics import roc_curve

fpr_reg_log_smote, tpr_reg_log_smote, _ = roc_curve(y_test, y_proba_reg_log_smote)
fpr_reg_log_oversampler, tpr_reg_log_oversampler, _ = roc_curve(y_test, y_proba_reg_log_oversampler)
fpr_reg_log_adasyn, tpr_reg_log_adasyn, _ = roc_curve(y_test, y_proba_reg_log_adasyn)
fpr_reg_log_bordeline_smote, tpr_reg_log_bordeline_smote, _ = roc_curve(y_test, y_proba_reg_log_bordeline_smote)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(fpr_reg_log_smote, tpr_reg_log_smote, label='Régression logistique avec SMOTE')
plt.plot(fpr_reg_log_oversampler, tpr_reg_log_oversampler, label='Régression logistique avec RandomOverSampler')
plt.plot(fpr_reg_log_adasyn, tpr_reg_log_adasyn, label='Régression logistique avec ADASYN')
plt.plot(fpr_reg_log_bordeline_smote, tpr_reg_log_bordeline_smote, label='Régression logistique avec BorderlineSMOTE')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC')
plt.legend()

plt.show()

In [ ]:
# Comparaison des courbes PR

from sklearn.metrics import precision_recall_curve

precision_reg_log_smote, recall_reg_log_smote, _ = precision_recall_curve(y_test, y_proba_reg_log_smote)
precision_reg_log_oversampler, recall_reg_log_oversampler, _ = precision_recall_curve(y_test, y_proba_reg_log_oversampler)
precision_reg_log_adasyn, recall_reg_log_adasyn, _ = precision_recall_curve(y_test, y_proba_reg_log_adasyn)
precision_reg_log_bordeline_smote, recall_reg_log_bordeline_smote, _ = precision_recall_curve(y_test, y_proba_reg_log_bordeline_smote)

plt.figure(figsize=(10, 6))
plt.plot(recall_reg_log_smote, precision_reg_log_smote, label='Régression logistique avec SMOTE')
plt.plot(recall_reg_log_oversampler, precision_reg_log_oversampler, label='Régression logistique avec RandomOverSampler')
plt.plot(recall_reg_log_adasyn, precision_reg_log_adasyn, label='Régression logistique avec ADASYN')
plt.plot(recall_reg_log_bordeline_smote, precision_reg_log_bordeline_smote, label='Régression logistique avec BorderlineSMOTE')
plt.xlabel('Rappel')
plt.ylabel('Précision')
plt.title('Courbes PR')
plt.legend()
plt.show()

In [ ]:
# Résultats 

resultats = pd.DataFrame({
    "AUC-ROC": [auc_roc_reg_log_smote, auc_roc_reg_log_oversampler, auc_roc_reg_log_adasyn, auc_roc_reg_log_borderline_smote],
    "AUC-PR": [auc_pr_reg_log_smote, auc_pr_reg_log_oversampler, auc_pr_reg_log_adasyn, auc_pr_reg_log_borderline_smote],
    "Log Loss": [log_loss_value_reg_log_smote, log_loss_value_reg_log_oversampler, log_loss_value_reg_log_adasyn, log_loss_value_reg_log_borderline_smote],
   # "Précision": [precision_reg_log_smote, precision_reg_log_oversampler, precision_reg_log_adasyn, precision_reg_log_borderline_smote],
    #"Rappel": [recall_reg_log_smote, recall_reg_log_oversampler, recall_reg_log_adasyn, recall_reg_log_borderline_smote],
    "F1 Score": [f1_reg_log_smote, f1_reg_log_oversampler, f1_reg_log_adasyn, f1_reg_log_borderline_smote],
    "MCC": [mcc_reg_log_smote, mcc_reg_log_oversampler, mcc_reg_log_adasyn, mcc_reg_log_borderline_smote],
    "Accuracy": [balanced_acc_reg_log_smote, balanced_acc_reg_log_oversampler, balanced_acc_reg_log_adasyn, balanced_acc_reg_log_borderline_smote],
    "Spécificité": [specificity_reg_log_smote, specificity_reg_log_oversampler, specificity_reg_log_adasyn, specificity_reg_log_borderline_smote],
    "Kappa de Cohen": [cohen_kappa_reg_log_smote, cohen_kappa_reg_log_oversampler, cohen_kappa_reg_log_adasyn, cohen_kappa_reg_log_borderline_smote]
}, index=["Reg log - SMOTE", "Reg log - RandomOverSampler", "Reg log - ADASYN", "Reg log - BorderlineSMOTE"])

# Arrondir les résultats à 3 chiffres après la virgule
resultats = resultats.round(3)

resultats

#### C) Ajustement des hyperparamètres

Pour la régression logistique, les hyperparamètres les plus courants à optimiser sont :
- C : Ce paramètre de régularisation inverse. Une valeur plus petite indique une régularisation plus forte.
- solver : L'algorithme utilisé pour l'optimisation. Les choix courants sont 'liblinear', 'lbfgs', 'saga', etc.
- penalty : Le type de régularisation à utiliser ('l1', 'l2', 'elasticnet', 'none')
- fonction de lien 

In [ ]:
# Ajustement du modèle

from sklearn.model_selection import GridSearchCV

# Définir les paramètres de la grille
grille_param = {
    'reg_log__C': [0.01, 0.1, 1, 10, 100], # Inverse de la force de régularisation : pas d'utilité pour la régression logistique sans régularisation
    'reg_log__penalty': ['l1', 'l2', 'elasticnet', 'none'], # Type de régularisation
}

# Création du GridSearchCV
grid_search = GridSearchCV(modele_reg_log_adasyn, grille_param, cv=5)

# Entraînement
grid_search.fit(X_train, y_train)

In [ ]:
# Meilleurs paramètres
print(f"Meilleurs paramètres : {grid_search.best_params_}")

In [ ]:
# Meilleur modèle
best_model_reg_log = grid_search.best_estimator_
best_model_reg_log

In [ ]:
# Prévisions
y_proba_reg_log_cv = modele_reg_log.predict_proba(X_test)[:, 1]

#### D) Mesure de la performance des modèles optimisés

In [ ]:
auc_roc = roc_auc_score(y_test, y_proba_reg_log_cv)
print(f"AUC-ROC: {auc_roc}")

In [ ]:
# Comparaison des courbes ROC

fpr_reg_log_cv, tpr_reg_log_cv, _ = roc_curve(y_test, y_proba_reg_log_cv)


## I.2.2. Abres de décision

#### A) Construction des modèles

In [ ]:
# Construction du modèle d'arbres de décision avec rééquilibrage des classes

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

etapes_arbre_oversampler = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('oversampler', RandomOverSampler()), # Sur-échantillonnage
        ('arbre', DecisionTreeClassifier()), # Arbre de décision
        ]

etapes_arbre_smote = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('smote', SMOTE()), # Sur-échantillonnage
        ('arbre', DecisionTreeClassifier()), # Arbre de décision
        ]

etapes_arbre_adasyn = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('adasyn', ADASYN()), # Sur-échantillonnage
        ('arbre', DecisionTreeClassifier()), # Arbre de décision
        ]

etapes_arbre_bordeline_smote = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('bordeline_smote', BorderlineSMOTE()), # Sur-échantillonnage
        ('arbre', DecisionTreeClassifier()), # Arbre de décision
        ]



modele_arbre_oversampler = Pipeline(steps=etapes_arbre_oversampler) # Création du pipeline
modele_arbre_smote = Pipeline(steps=etapes_arbre_smote) # Création du pipeline
modele_arbre_adasyn = Pipeline(steps=etapes_arbre_adasyn) # Création du pipeline
modele_arbre_bordeline_smote = Pipeline(steps=etapes_arbre_bordeline_smote) # Création du pipeline)

In [ ]:
modele_arbre_oversampler.fit(X_train, y_train) # Entraînement

In [ ]:
modele_arbre_smote.fit(X_train, y_train) # Entraînement

In [ ]:
modele_arbre_adasyn.fit(X_train, y_train) # Entraînement

In [ ]:
modele_arbre_bordeline_smote.fit(X_train, y_train) # Entraînement

In [ ]:
# Prédictions

y_proba_arbre_oversampler = modele_arbre_oversampler.predict_proba(X_test)[:, 1]
y_pred_arbre_oversampler = modele_arbre_oversampler.predict(X_test)

y_proba_arbre_smote = modele_arbre_smote.predict_proba(X_test)[:, 1]
y_pred_arbre_smote = modele_arbre_smote.predict(X_test)

y_proba_arbre_adasyn = modele_arbre_adasyn.predict_proba(X_test)[:, 1]
y_pred_arbre_adasyn = modele_arbre_adasyn.predict(X_test)

y_proba_arbre_bordeline_smote = modele_arbre_bordeline_smote.predict_proba(X_test)[:, 1]
y_pred_arbre_bordeline_smote = modele_arbre_bordeline_smote.predict(X_test)

#### B) Mesure de la performance des modèles initiaux

In [ ]:
# Calcul des métriques

from sklearn.metrics import roc_auc_score, average_precision_score, log_loss
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, cohen_kappa_score

auc_roc_arbre_oversampler = roc_auc_score(y_test, y_proba_arbre_oversampler)
auc_pr_arbre_oversampler = average_precision_score(y_test, y_proba_arbre_oversampler)
log_loss_value_arbre_oversampler = log_loss(y_test, y_proba_arbre_oversampler)
precision_arbre_oversampler = precision_score(y_test, y_pred_arbre_oversampler)
recall_arbre_oversampler = recall_score(y_test, y_pred_arbre_oversampler)
f1_arbre_oversampler = f1_score(y_test, y_pred_arbre_oversampler)
mcc_arbre_oversampler = matthews_corrcoef(y_test, y_pred_arbre_oversampler)
balanced_acc_arbre_oversampler = balanced_accuracy_score(y_test, y_pred_arbre_oversampler)
specificity_arbre_oversampler = confusion_matrix(y_test, y_pred_arbre_oversampler)[0, 0] / (confusion_matrix(y_test, y_pred_arbre_oversampler)[0, 0] + confusion_matrix(y_test, y_pred_arbre_oversampler)[0, 1])
cohen_kappa_arbre_oversampler = cohen_kappa_score(y_test, y_pred_arbre_oversampler)

In [ ]:
auc_roc_arbre_smote = roc_auc_score(y_test, y_proba_arbre_smote)
auc_pr_arbre_smote = average_precision_score(y_test, y_proba_arbre_smote)
log_loss_value_arbre_smote = log_loss(y_test, y_proba_arbre_smote)
precision_arbre_smote = precision_score(y_test, y_pred_arbre_smote)
recall_arbre_smote = recall_score(y_test, y_pred_arbre_smote)
f1_arbre_smote = f1_score(y_test, y_pred_arbre_smote)
mcc_arbre_smote = matthews_corrcoef(y_test, y_pred_arbre_smote)
balanced_acc_arbre_smote = balanced_accuracy_score(y_test, y_pred_arbre_smote)
specificity_arbre_smote = confusion_matrix(y_test, y_pred_arbre_smote)[0, 0] / (confusion_matrix(y_test, y_pred_arbre_smote)[0, 0] + confusion_matrix(y_test, y_pred_arbre_smote)[0, 1])
cohen_kappa_arbre_smote = cohen_kappa_score(y_test, y_pred_arbre_smote)

In [ ]:
auc_roc_arbre_adasyn = roc_auc_score(y_test, y_proba_arbre_adasyn)
auc_pr_arbre_adasyn = average_precision_score(y_test, y_proba_arbre_adasyn)
log_loss_value_arbre_adasyn = log_loss(y_test, y_proba_arbre_adasyn)
precision_arbre_adasyn = precision_score(y_test, y_pred_arbre_adasyn)
recall_arbre_adasyn = recall_score(y_test, y_pred_arbre_adasyn)
f1_arbre_adasyn = f1_score(y_test, y_pred_arbre_adasyn)
mcc_arbre_adasyn = matthews_corrcoef(y_test, y_pred_arbre_adasyn)
balanced_acc_arbre_adasyn = balanced_accuracy_score(y_test, y_pred_arbre_adasyn)
specificity_arbre_adasyn = confusion_matrix(y_test, y_pred_arbre_adasyn)[0, 0] / (confusion_matrix(y_test, y_pred_arbre_adasyn)[0, 0] + confusion_matrix(y_test, y_pred_arbre_adasyn)[0, 1])
cohen_kappa_arbre_adasyn = cohen_kappa_score(y_test, y_pred_arbre_adasyn)

In [ ]:
auc_roc_arbre_bordeline_smote = roc_auc_score(y_test, y_proba_arbre_bordeline_smote)
auc_pr_arbre_bordeline_smote = average_precision_score(y_test, y_proba_arbre_bordeline_smote)
log_loss_value_arbre_bordeline_smote = log_loss(y_test, y_proba_arbre_bordeline_smote)
precision_arbre_bordeline_smote = precision_score(y_test, y_pred_arbre_bordeline_smote)
recall_arbre_bordeline_smote = recall_score(y_test, y_pred_arbre_bordeline_smote)
f1_arbre_bordeline_smote = f1_score(y_test, y_pred_arbre_bordeline_smote)
mcc_arbre_bordeline_smote = matthews_corrcoef(y_test, y_pred_arbre_bordeline_smote)
balanced_acc_arbre_bordeline_smote = balanced_accuracy_score(y_test, y_pred_arbre_bordeline_smote)
specificity_arbre_bordeline_smote = confusion_matrix(y_test, y_pred_arbre_bordeline_smote)[0, 0] / (confusion_matrix(y_test, y_pred_arbre_bordeline_smote)[0, 0] + confusion_matrix(y_test, y_pred_arbre_bordeline_smote)[0, 1])
cohen_kappa_arbre_bordeline_smote = cohen_kappa_score(y_test, y_pred_arbre_bordeline_smote)

In [ ]:
# Affichage des résultats
resultats = pd.DataFrame({
    "AUC-ROC": [auc_roc_arbre_oversampler, auc_roc_arbre_smote, auc_roc_arbre_adasyn, auc_roc_arbre_bordeline_smote],
    "AUC-PR": [auc_pr_arbre_oversampler, auc_pr_arbre_smote, auc_pr_arbre_adasyn, auc_pr_arbre_bordeline_smote],
    "Log Loss": [log_loss_value_arbre_oversampler, log_loss_value_arbre_smote, log_loss_value_arbre_adasyn, log_loss_value_arbre_bordeline_smote],
    #"Précision": [precision_arbre_oversampler, precision_arbre_smote, precision_arbre_adasyn, precision_arbre_bordeline_smote],
    #"Rappel": [recall_arbre_oversampler, recall_arbre_smote, recall_arbre_adasyn, recall_arbre_bordeline_smote],
    "F1 Score": [f1_arbre_oversampler, f1_arbre_smote, f1_arbre_adasyn, f1_arbre_bordeline_smote],
    "MCC": [mcc_arbre_oversampler, mcc_arbre_smote, mcc_arbre_adasyn, mcc_arbre_bordeline_smote],
    "Accuracy": [balanced_acc_arbre_oversampler, balanced_acc_arbre_smote, balanced_acc_arbre_adasyn, balanced_acc_arbre_bordeline_smote],
    "Spécificité": [specificity_arbre_oversampler, specificity_arbre_smote, specificity_arbre_adasyn, specificity_arbre_bordeline_smote],
    "Kappa de Cohen": [cohen_kappa_arbre_oversampler, cohen_kappa_arbre_smote, cohen_kappa_arbre_adasyn, cohen_kappa_arbre_bordeline_smote]
}, index=["Arbre - RandomOverSampler", "Arbre - SMOTE", "Arbre - ADASYN", "Arbre - BorderlineSMOTE"])

# Arrondir les résultats à 3 chiffres après la virgule
resultats = resultats.round(3)
resultats

In [ ]:
# Tracé des courbes ROC

from sklearn.metrics import roc_curve

fpr_arbre_oversampler, tpr_arbre_oversampler, _ = roc_curve(y_test, y_proba_arbre_oversampler)
fpr_arbre_smote, tpr_arbre_smote, _ = roc_curve(y_test, y_proba_arbre_smote)
fpr_arbre_adasyn, tpr_arbre_adasyn, _ = roc_curve(y_test, y_proba_arbre_adasyn)
fpr_arbre_bordeline_smote, tpr_arbre_bordeline_smote, _ = roc_curve(y_test, y_proba_arbre_bordeline_smote)

plt.figure(figsize=(10, 6))
plt.plot(fpr_arbre_oversampler, tpr_arbre_oversampler, label='Arbre avec RandomOverSampler')
plt.plot(fpr_arbre_smote, tpr_arbre_smote, label='Arbre avec SMOTE')
plt.plot(fpr_arbre_adasyn, tpr_arbre_adasyn, label='Arbre avec ADASYN')
plt.plot(fpr_arbre_bordeline_smote, tpr_arbre_bordeline_smote, label='Arbre avec BorderlineSMOTE')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC')
plt.legend()
plt.show()

#### C) Ajustement des hyperparamètres

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score

# Définition de la grille des hyperparamètres pour l'arbre de décision
param_grid = {
    'arbre__criterion': ['gini', 'entropy', 'log_loss'], # Critère d'impureté 
    'arbre__max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], # Profondeur maximale de l'arbre
    'arbre__min_samples_split': [2, 5, 10, 20,30], # Nombre minimum d'échantillons pour diviser un nœud
    'arbre__min_samples_leaf': [1, 2, 4] # Nombre minimum d'échantillons requis à chaque feuille
}

# Choix des métriques adaptées
scoring = {
    'F1': make_scorer(f1_score), 
    'Recall': make_scorer(recall_score),
    'AUC': 'roc_auc'
}

In [ ]:
pipelines = {
    'oversampler': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('oversampler', RandomOverSampler()), 
        ('arbre', DecisionTreeClassifier())
    ]),
    'smote': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('smote', SMOTE()), 
        ('arbre', DecisionTreeClassifier())
    ]),
    'adasyn': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('adasyn', ADASYN()), 
        ('arbre', DecisionTreeClassifier())
    ]),
    'bordeline_smote': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('bordeline_smote', BorderlineSMOTE()), 
        ('arbre', DecisionTreeClassifier())
    ])
}

# On stocke les meilleurs paramètres pour chaque modèle
best_params = {}
for name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grid, scoring=scoring, refit='F1', cv=5)
    grid_search.fit(X_train, y_train)
    best_params[name] = grid_search.best_params_
    print(f"Meilleurs paramètres pour {name} : {grid_search.best_params_}")

# On stocke les meilleurs modèles
best_models = {}
for name, pipeline in pipelines.items():
    best_models[name] = grid_search.best_estimator_

In [ ]:
# Prédictions
y_proba_arbre_oversampler_cv = best_models['oversampler'].predict_proba(X_test)[:, 1]
y_proba_arbre_smote_cv = best_models['smote'].predict_proba(X_test)[:, 1]
y_proba_arbre_adasyn_cv = best_models['adasyn'].predict_proba(X_test)[:, 1]
y_proba_arbre_bordeline_smote_cv = best_models['bordeline_smote'].predict_proba(X_test)[:, 1]

#### D) Mesure de la performance des modèles optimisés

In [ ]:
# Calcul des métriques

auc_roc_arbre_oversampler_cv = roc_auc_score(y_test, y_proba_arbre_oversampler_cv)
auc_pr_arbre_oversampler_cv = average_precision_score(y_test, y_proba_arbre_oversampler_cv)
log_loss_value_arbre_oversampler_cv = log_loss(y_test, y_proba_arbre_oversampler_cv)
precision_arbre_oversampler_cv = precision_score(y_test, y_proba_arbre_oversampler_cv)
recall_arbre_oversampler_cv = recall_score(y_test, y_proba_arbre_oversampler_cv)
f1_arbre_oversampler_cv = f1_score(y_test, y_proba_arbre_oversampler_cv)
mcc_arbre_oversampler_cv = matthews_corrcoef(y_test, y_proba_arbre_oversampler_cv)
balanced_acc_arbre_oversampler_cv = balanced_accuracy_score(y_test, y_proba_arbre_oversampler_cv)

auc_roc_arbre_smote_cv = roc_auc_score(y_test, y_proba_arbre_smote_cv)
auc_pr_arbre_smote_cv = average_precision_score(y_test, y_proba_arbre_smote_cv)
log_loss_value_arbre_smote_cv = log_loss(y_test, y_proba_arbre_smote_cv)
precision_arbre_smote_cv = precision_score(y_test, y_proba_arbre_smote_cv)
recall_arbre_smote_cv = recall_score(y_test, y_proba_arbre_smote_cv)
f1_arbre_smote_cv = f1_score(y_test, y_proba_arbre_smote_cv)
mcc_arbre_smote_cv = matthews_corrcoef(y_test, y_proba_arbre_smote_cv)
balanced_acc_arbre_smote_cv = balanced_accuracy_score(y_test, y_proba_arbre_smote_cv)

auc_roc_arbre_adasyn_cv = roc_auc_score(y_test, y_proba_arbre_adasyn_cv)
auc_pr_arbre_adasyn_cv = average_precision_score(y_test, y_proba_arbre_adasyn_cv)
log_loss_value_arbre_adasyn_cv = log_loss(y_test, y_proba_arbre_adasyn_cv)
precision_arbre_adasyn_cv = precision_score(y_test, y_proba_arbre_adasyn_cv)
recall_arbre_adasyn_cv = recall_score(y_test, y_proba_arbre_adasyn_cv)
f1_arbre_adasyn_cv = f1_score(y_test, y_proba_arbre_adasyn_cv)
mcc_arbre_adasyn_cv = matthews_corrcoef(y_test, y_proba_arbre_adasyn_cv)
balanced_acc_arbre_adasyn_cv = balanced_accuracy_score(y_test, y_proba_arbre_adasyn_cv)

auc_roc_arbre_bordeline_smote_cv = roc_auc_score(y_test, y_proba_arbre_bordeline_smote_cv)
auc_pr_arbre_bordeline_smote_cv = average_precision_score(y_test, y_proba_arbre_bordeline_smote_cv)
log_loss_value_arbre_bordeline_smote_cv = log_loss(y_test, y_proba_arbre_bordeline_smote_cv)
precision_arbre_bordeline_smote_cv = precision_score(y_test, y_proba_arbre_bordeline_smote_cv)
recall_arbre_bordeline_smote_cv = recall_score(y_test, y_proba_arbre_bordeline_smote_cv)
f1_arbre_bordeline_smote_cv = f1_score(y_test, y_proba_arbre_bordeline_smote_cv)
mcc_arbre_bordeline_smote_cv = matthews_corrcoef(y_test, y_proba_arbre_bordeline_smote_cv)
balanced_acc_arbre_bordeline_smote_cv = balanced_accuracy_score(y_test, y_proba_arbre_bordeline_smote_cv)


In [2]:
# Affichage propre des résultats

resultats = pd.DataFrame({
    "AUC-ROC": [auc_roc_arbre_oversampler_cv, auc_roc_arbre_smote_cv, auc_roc_arbre_adasyn_cv, auc_roc_arbre_bordeline_smote_cv],
    "AUC-PR": [auc_pr_arbre_oversampler_cv, auc_pr_arbre_smote_cv, auc_pr_arbre_adasyn_cv, auc_pr_arbre_bordeline_smote_cv],
    "Log Loss": [log_loss_value_arbre_oversampler_cv, log_loss_value_arbre_smote_cv, log_loss_value_arbre_adasyn_cv, log_loss_value_arbre_bordeline_smote_cv],
    "Précision": [precision_arbre_oversampler_cv, precision_arbre_smote_cv, precision_arbre_adasyn_cv, precision_arbre_bordeline_smote_cv],
    "Rappel": [recall_arbre_oversampler_cv, recall_arbre_smote_cv, recall_arbre_adasyn_cv, recall_arbre_bordeline_smote_cv],
    "F1 Score": [f1_arbre_oversampler_cv, f1_arbre_smote_cv, f1_arbre_adasyn_cv, f1_arbre_bordeline_smote_cv],
    "MCC": [mcc_arbre_oversampler_cv, mcc_arbre_smote_cv, mcc_arbre_adasyn_cv, mcc_arbre_bordeline_smote_cv],
    "Accuracy": [balanced_acc_arbre_oversampler_cv, balanced_acc_arbre_smote_cv, balanced_acc_arbre_adasyn_cv, balanced_acc_arbre_bordeline_smote_cv]
}, index=["Arbre - RandomOverSampler", "Arbre - SMOTE", "Arbre - ADASYN", "Arbre - BorderlineSMOTE"])

# Arrondir les résultats à 3 chiffres après la virgule
resultats = resultats.round(3)
resultats

NameError: name 'pd' is not defined

### I.2.3. Forêts Aléatoires 

#### A) Construction des modèles

In [ ]:
# Construction du modèle de forêts aléatoires avec rééquilibrage des classes

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

etapes_rf_oversampler = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('oversampler', RandomOverSampler()), # Sur-échantillonnage
        ('arbre', RandomForestClassifier()), # Forets aléatoires
        ]

etapes_rf_smote = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('smote', SMOTE()), # Sur-échantillonnage
        ('arbre', RandomForestClassifier()), # Forets aléatoires
        ]

etapes_rf_adasyn = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('adasyn', ADASYN()), # Sur-échantillonnage
        ('arbre', RandomForestClassifier()), # Forets aléatoires
        ]

etapes_rf_bordeline_smote = [('std_scaler', StandardScaler()), # Standardisation des variables
        ('bordeline_smote', BorderlineSMOTE()), # Sur-échantillonnage
        ('rf', RandomForestClassifier()), # Forets aléatoires
        ]



modele_rf_oversampler = Pipeline(steps=etapes_rf_oversampler) # Création du pipeline
modele_rf_smote = Pipeline(steps=etapes_rf_smote) # Création du pipeline
modele_rf_adasyn = Pipeline(steps=etapes_rf_adasyn) # Création du pipeline
modele_rf_bordeline_smote = Pipeline(steps=etapes_rf_bordeline_smote) # Création du pipeline)

modele_rf_oversampler.fit(X_train, y_train) # Entraînement
modele_rf_smote.fit(X_train, y_train) # Entraînement
modele_rf_adasyn.fit(X_train, y_train) # Entraînement
modele_rf_bordeline_smote.fit(X_train, y_train) # Entraînement

In [ ]:
# Prédictions

y_proba_rf_oversampler = modele_rf_oversampler.predict_proba(X_test)[:, 1]
y_pred_rf_oversampler = modele_rf_oversampler.predict(X_test)

y_proba_rf_smote = modele_rf_smote.predict_proba(X_test)[:, 1]
y_pred_rf_smote = modele_rf_smote.predict(X_test)

y_proba_rf_adasyn = modele_rf_adasyn.predict_proba(X_test)[:, 1]
y_pred_rf_adasyn = modele_rf_adasyn.predict(X_test)

y_proba_rf_bordeline_smote = modele_rf_bordeline_smote.predict_proba(X_test)[:, 1]
y_pred_rf_bordeline_smote = modele_rf_bordeline_smote.predict(X_test)

#### B) Mesure de la performance des modèles initiaux

In [3]:
# Calcul des métriques

from sklearn.metrics import roc_auc_score, average_precision_score, log_loss
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, cohen_kappa_score

auc_roc_rf_oversampler = roc_auc_score(y_test, y_proba_rf_oversampler)
auc_pr_rf_oversampler = average_precision_score(y_test, y_proba_rf_oversampler)
log_loss_value_rf_oversampler = log_loss(y_test, y_proba_rf_oversampler)
precision_rf_oversampler = precision_score(y_test, y_pred_rf_oversampler)
recall_rf_oversampler = recall_score(y_test, y_pred_rf_oversampler)
f1_rf_oversampler = f1_score(y_test, y_pred_rf_oversampler)
mcc_rf_oversampler = matthews_corrcoef(y_test, y_pred_rf_oversampler)
balanced_acc_rf_oversampler = balanced_accuracy_score(y_test, y_pred_rf_oversampler)
specificity_rf_oversampler = confusion_matrix(y_test, y_pred_rf_oversampler)[0, 0] / (confusion_matrix(y_test, y_pred_rf_oversampler)[0, 0] + confusion_matrix(y_test, y_pred_rf_oversampler)[0, 1])
cohen_kappa_rf_oversampler = cohen_kappa_score(y_test, y_pred_rf_oversampler)

auc_roc_rf_smote = roc_auc_score(y_test, y_proba_rf_smote)
auc_pr_rf_smote = average_precision_score(y_test, y_proba_rf_smote)
log_loss_value_rf_smote = log_loss(y_test, y_proba_rf_smote)
precision_rf_smote = precision_score(y_test, y_pred_rf_smote)
recall_rf_smote = recall_score(y_test, y_pred_rf_smote)
f1_rf_smote = f1_score(y_test, y_pred_rf_smote)
mcc_rf_smote = matthews_corrcoef(y_test, y_pred_rf_smote)
balanced_acc_rf_smote = balanced_accuracy_score(y_test, y_pred_rf_smote)
specificity_rf_smote = confusion_matrix(y_test, y_pred_rf_smote)[0, 0] / (confusion_matrix(y_test, y_pred_rf_smote)[0, 0] + confusion_matrix(y_test, y_pred_rf_smote)[0, 1])
cohen_kappa_rf_smote = cohen_kappa_score(y_test, y_pred_rf_smote)

auc_roc_rf_adasyn = roc_auc_score(y_test, y_proba_rf_adasyn)
auc_pr_rf_adasyn = average_precision_score(y_test, y_proba_rf_adasyn)
log_loss_value_rf_adasyn = log_loss(y_test, y_proba_rf_adasyn)
precision_rf_adasyn = precision_score(y_test, y_pred_rf_adasyn)
recall_rf_adasyn = recall_score(y_test, y_pred_rf_adasyn)
f1_rf_adasyn = f1_score(y_test, y_pred_rf_adasyn)
mcc_rf_adasyn = matthews_corrcoef(y_test, y_pred_rf_adasyn)
balanced_acc_rf_adasyn = balanced_accuracy_score(y_test, y_pred_rf_adasyn)
specificity_rf_adasyn = confusion_matrix(y_test, y_pred_rf_adasyn)[0, 0] / (confusion_matrix(y_test, y_pred_rf_adasyn)[0, 0] + confusion_matrix(y_test, y_pred_rf_adasyn)[0, 1])
cohen_kappa_rf_adasyn = cohen_kappa_score(y_test, y_pred_rf_adasyn)

auc_roc_rf_bordeline_smote = roc_auc_score(y_test, y_proba_rf_bordeline_smote)
auc_pr_rf_bordeline_smote = average_precision_score(y_test, y_proba_rf_bordeline_smote)
log_loss_value_rf_bordeline_smote = log_loss(y_test, y_proba_rf_bordeline_smote)
precision_rf_bordeline_smote = precision_score(y_test, y_pred_rf_bordeline_smote)
recall_rf_bordeline_smote = recall_score(y_test, y_pred_rf_bordeline_smote)
f1_rf_bordeline_smote = f1_score(y_test, y_pred_rf_bordeline_smote)
mcc_rf_bordeline_smote = matthews_corrcoef(y_test, y_pred_rf_bordeline_smote)
balanced_acc_rf_bordeline_smote = balanced_accuracy_score(y_test, y_pred_rf_bordeline_smote)
specificity_rf_bordeline_smote = confusion_matrix(y_test, y_pred_rf_bordeline_smote)[0, 0] / (confusion_matrix(y_test, y_pred_rf_bordeline_smote)[0, 0] + confusion_matrix(y_test, y_pred_rf_bordeline_smote)[0, 1])
cohen_kappa_rf_bordeline_smote = cohen_kappa_score(y_test, y_pred_rf_bordeline_smote)

SyntaxError: incomplete input (764029348.py, line 9)

In [ ]:
# Affichage des résultats

resultats = pd.DataFrame({
    "AUC-ROC": [auc_roc_rf_oversampler, auc_roc_rf_smote, auc_roc_rf_adasyn, auc_roc_rf_bordeline_smote],
    "AUC-PR": [auc_pr_rf_oversampler, auc_pr_rf_smote, auc_pr_rf_adasyn, auc_pr_rf_bordeline_smote],
    "Log Loss": [log_loss_value_rf_oversampler, log_loss_value_rf_smote, log_loss_value_rf_adasyn, log_loss_value_rf_bordeline_smote],
    #"Précision": [precision_rf_oversampler, precision_rf_smote, precision_rf_adasyn, precision_rf_bordeline_smote],
    #"Rappel": [recall_rf_oversampler, recall_rf_smote, recall_rf_adasyn, recall_rf_bordeline_smote],
    "F1 Score": [f1_rf_oversampler, f1_rf_smote, f1_rf_adasyn, f1_rf_bordeline_smote],
    "MCC": [mcc_rf_oversampler, mcc_rf_smote, mcc_rf_adasyn, mcc_rf_bordeline_smote],
    "Accuracy": [balanced_acc_rf_oversampler, balanced_acc_rf_smote, balanced_acc_rf_adasyn, balanced_acc_rf_bordeline_smote],
    "Spécificité": [specificity_rf_oversampler, specificity_rf_smote, specificity_rf_adasyn, specificity_rf_bordeline_smote],
    "Kappa de Cohen": [cohen_kappa_rf_oversampler, cohen_kappa_rf_smote, cohen_kappa_rf_adasyn, cohen_kappa_rf_bordeline_smote]
}, index=["RF - RandomOverSampler", "RF - SMOTE", "RF - ADASYN", "RF - BorderlineSMOTE"])

# Arrondir les résultats à 3 chiffres après la virgule
resultats = resultats.round(3)
resultats

In [4]:
# Courbes ROC

from sklearn.metrics import roc_curve

fpr_rf_oversampler, tpr_rf_oversampler, _ = roc_curve(y_test, y_proba_rf_oversampler)
fpr_rf_smote, tpr_rf_smote, _ = roc_curve(y_test, y_proba_rf_smote)
fpr_rf_adasyn, tpr_rf_adasyn, _ = roc_curve(y_test, y_proba_rf_adasyn)
fpr_rf_bordeline_smote, tpr_rf_bordeline_smote, _ = roc_curve(y_test, y_proba_rf_bordeline_smote)

plt.figure(figsize=(10, 6))
plt.plot(fpr_rf_oversampler, tpr_rf_oversampler, label='RF avec RandomOverSampler')
plt.plot(fpr_rf_smote, tpr_rf_smote, label='RF avec SMOTE')
plt.plot(fpr_rf_adasyn, tpr_rf_adasyn, label='RF avec ADASYN')
plt.plot(fpr_rf_bordeline_smote, tpr_rf_bordeline_smote, label='RF avec BorderlineSMOTE')
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbes ROC')
plt.legend()
plt.show()

NameError: name 'y_test' is not defined

#### C) Ajustement des hyperparamètres

In [ ]:
# Ajustement des hyperparamètres par validation croisée

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, recall_score

# Définition de la grille des hyperparamètres pour les forêts aléatoires

param_grid = {
    'rf__n_estimators': [100, 200, 300, 400, 500], # Nombre d'arbres
    'rf__criterion': ['gini', 'entropy'], # Critère d'impureté
    'rf__max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], # Profondeur maximale de l'arbre
    'rf__min_samples_split': [2, 5, 10, 20, 30], # Nombre minimum d'échantillons pour diviser un nœud
    'rf__min_samples_leaf': [1, 2, 4] # Nombre minimum d'échantillons requis à chaque feuille
}

# Choix des métriques adaptées
scoring = {
    'F1': make_scorer(f1_score), 
    'Recall': make_scorer(recall_score),
    'AUC': 'roc_auc'
}

In [ ]:
# Pipelines

pipelines = {
    'oversampler': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('oversampler', RandomOverSampler()), 
        ('rf', RandomForestClassifier())
    ]),
    'smote': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('smote', SMOTE()), 
        ('rf', RandomForestClassifier())
    ]),
    'adasyn': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('adasyn', ADASYN()), 
        ('rf', RandomForestClassifier())
    ]),
    'bordeline_smote': Pipeline([
        ('std_scaler', StandardScaler()), 
        ('bordeline_smote', BorderlineSMOTE()), 
        ('rf', RandomForestClassifier())
    ])
}

# On stocke les meilleurs paramètres pour chaque modèle
best_params = {}
for name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grid, scoring=scoring, refit='F1', cv=5)
    grid_search.fit(X_train, y_train)
    best_params[name] = grid_search.best_params_
    print(f"Meilleurs paramètres pour {name} : {grid_search.best_params_}")

# On stocke les meilleurs modèles
best_models = {}
for name, pipeline in pipelines.items():
    best_models[name] = grid_search.best_estimator_

In [ ]:
# Prédictions

y_proba_rf_oversampler_cv = best_models['oversampler'].predict_proba(X_test)[:, 1]
y_proba_rf_smote_cv = best_models['smote'].predict_proba(X_test)[:, 1]
y_proba_rf_adasyn_cv = best_models['adasyn'].predict_proba(X_test)[:, 1]
y_proba_rf_bordeline_smote_cv = best_models['bordeline_smote'].predict_proba(X_test)[:, 1]

#### D) Mesure de la performance des modèles optimisés

In [ ]:
# Calcul des métriques

auc_roc_rf_oversampler_cv = roc_auc_score(y_test, y_proba_rf_oversampler_cv)
auc_pr_rf_oversampler_cv = average_precision_score(y_test, y_proba_rf_oversampler_cv)
log_loss_value_rf_oversampler_cv = log_loss(y_test, y_proba_rf_oversampler_cv)
precision_rf_oversampler_cv = precision_score(y_test, y_proba_rf_oversampler_cv)
recall_rf_oversampler_cv = recall_score(y_test, y_proba_rf_oversampler_cv)
f1_rf_oversampler_cv = f1_score(y_test, y_proba_rf_oversampler_cv)
mcc_rf_oversampler_cv = matthews_corrcoef(y_test, y_proba_rf_oversampler_cv)
balanced_acc_rf_oversampler_cv = balanced_accuracy_score(y_test, y_proba_rf_oversampler_cv)

auc_roc_rf_smote_cv = roc_auc_score(y_test, y_proba_rf_smote_cv)
auc_pr_rf_smote_cv = average_precision_score(y_test, y_proba_rf_smote_cv)
log_loss_value_rf_smote_cv = log_loss(y_test, y_proba_rf_smote_cv)
precision_rf_smote_cv = precision_score(y_test, y_proba_rf_smote_cv)
recall_rf_smote_cv = recall_score(y_test, y_proba_rf_smote_cv)
f1_rf_smote_cv = f1_score(y_test, y_proba_rf_smote_cv)
mcc_rf_smote_cv = matthews_corrcoef(y_test, y_proba_rf_smote_cv)
balanced_acc_rf_smote_cv = balanced_accuracy_score(y_test, y_proba_rf_smote_cv)

auc_roc_rf_adasyn_cv = roc_auc_score(y_test, y_proba_rf_adasyn_cv)
auc_pr_rf_adasyn_cv = average_precision_score(y_test, y_proba_rf_adasyn_cv)
log_loss_value_rf_adasyn_cv = log_loss(y_test, y_proba_rf_adasyn_cv)
precision_rf_adasyn_cv = precision_score(y_test, y_proba_rf_adasyn_cv)
recall_rf_adasyn_cv = recall_score(y_test, y_proba_rf_adasyn_cv)
f1_rf_adasyn_cv = f1_score(y_test, y_proba_rf_adasyn_cv)
mcc_rf_adasyn_cv = matthews_corrcoef(y_test, y_proba_rf_adasyn_cv)
balanced_acc_rf_adasyn_cv = balanced_accuracy_score(y_test, y_proba_rf_adasyn_cv)

auc_roc_rf_bordeline_smote_cv = roc_auc_score(y_test, y_proba_rf_bordeline_smote_cv)
auc_pr_rf_bordeline_smote_cv = average_precision_score(y_test, y_proba_rf_bordeline_smote_cv)
log_loss_value_rf_bordeline_smote_cv = log_loss(y_test, y_proba_rf_bordeline_smote_cv)
precision_rf_bordeline_smote_cv = precision_score(y_test, y_proba_rf_bordeline_smote_cv)
recall_rf_bordeline_smote_cv = recall_score(y_test, y_proba_rf_bordeline_smote_cv)
f1_rf_bordeline_smote_cv = f1_score(y_test, y_proba_rf_bordeline_smote_cv)
mcc_rf_bordeline_smote_cv = matthews_corrcoef(y_test, y_proba_rf_bordeline_smote_cv)
balanced_acc_rf_bordeline_smote_cv = balanced_accuracy_score(y_test, y_proba_rf_bordeline_smote_cv)

In [ ]:
# Affichage des résultats

resultats = pd.DataFrame({
    "AUC-ROC": [auc_roc_rf_oversampler_cv, auc_roc_rf_smote_cv, auc_roc_rf_adasyn_cv, auc_roc_rf_bordeline_smote_cv],
    "AUC-PR": [auc_pr_rf_oversampler_cv, auc_pr_rf_smote_cv, auc_pr_rf_adasyn_cv, auc_pr_rf_bordeline_smote_cv],
    "Log Loss": [log_loss_value_rf_oversampler_cv, log_loss_value_rf_smote_cv, log_loss_value_rf_adasyn_cv, log_loss_value_rf_bordeline_smote_cv],
    "Précision": [precision_rf_oversampler_cv, precision_rf_smote_cv, precision_rf_adasyn_cv, precision_rf_bordeline_smote_cv],
    "Rappel": [recall_rf_oversampler_cv, recall_rf_smote_cv, recall_rf_adasyn_cv, recall_rf_bordeline_smote_cv],
    "F1 Score": [f1_rf_oversampler_cv, f1_rf_smote_cv, f1_rf_adasyn_cv, f1_rf_bordeline_smote_cv],
    "MCC": [mcc_rf_oversampler_cv, mcc_rf_smote_cv, mcc_rf_adasyn_cv, mcc_rf_bordeline_smote_cv],
    "Accuracy": [balanced_acc_rf_oversampler_cv, balanced_acc_rf_smote_cv, balanced_acc_rf_adasyn_cv, balanced_acc_rf_bordeline_smote_cv]
}, index=["RF - RandomOverSampler", "RF - SMOTE", "RF - ADASYN", "RF - BorderlineSMOTE"])

# Arrondir les résultats à 3 chiffres après la virgule
resultats = resultats.round(3)
resultats

### I.2.4. Gradient Boosting

#### A) Construction du modèle

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Modèle de gradient boosting
etapes = ([("std_scaler", StandardScaler()),
              ("smote", SMOTE()),
              ("clf_boosting", GradientBoostingClassifier())
    ])

model_gradient_boosting = Pipeline(steps=etapes)

In [ ]:
model_gradient_boosting.fit(X_train, y_train)

In [ ]:
y_pred_gradient_boosting = model_gradient_boosting.predict(X_test)

#### B) Mesure de la performance

In [ ]:
score_gradient_boosting = modele_reg_log.score(X_test, y_test)
print("Le score du modèle est : ", score_gradient_boosting)

In [ ]:
# Calcul de la matrice de confusion
from sklearn.metrics import confusion_matrix

mat_conf_gradient_boosting = confusion_matrix(y_test, y_pred_gradient_boosting)
print("La matrice de confusion est : \n", mat_conf_gradient_boosting)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

modalites =[0,1] 
fig, ax = plt.subplots()
tick_marks = np.arange(len(modalites))
plt.xticks(tick_marks, modalites)
plt.yticks(tick_marks, modalites)

sns.heatmap(pd.DataFrame(mat_conf_gradient_boosting), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Matrice de confusion', y=1.1)
plt.ylabel('Valeur observée')
plt.xlabel('valeur prédite')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

y_pred_prob_gradient_boosting = model_gradient_boosting.predict_proba(X_test)[:, 1] # probabilité que la classe soit 1
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_gradient_boosting) # calcul des taux de faux positifs et vrais positifs
roc_auc = auc(fpr, tpr) # calcul de l'aire sous la courbe ROC

# Tracé de la courbe ROC
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--') # ligne en pointillés représentant la performance d'un classificateur aléatoire
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Taux de faux positif (1 - spécificité)")
plt.ylabel('Taux de vrai positif (sensibilité)')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Calcul du score AUC

from sklearn.metrics import roc_auc_score

score_auc_gradient_boosting = roc_auc_score(y_test, y_pred_prob_gradient_boosting)
score_auc_gradient_boosting

In [ ]:
# Calcul de la précision

from sklearn.metrics import precision_score

precision_gradient_boosting = precision_score(y_test, y_pred_gradient_boosting)
print("Precision:", precision_gradient_boosting)

In [ ]:
from sklearn.metrics import recall_score

recall_gradient_boosting = recall_score(y_test, y_pred_gradient_boosting)
print("Recall:", recall_gradient_boosting)

In [ ]:
from sklearn.metrics import f1_score

# Calculate and print F1-score
f1_gradient_boosting = f1_score(y_test, y_pred_gradient_boosting)
print("F1-Score:", f1_gradient_boosting)

In [ ]:
# Résumé du modèle de gradient boosting

print("Modèle de gradient boosting \n")
print("Score AUC:", score_auc_gradient_boosting) 
print("F1-Score:", f1_gradient_boosting)

### I.2.5. Régression ridge

#### A) Construction du modèle

In [ ]:
# Régression ridge

from sklearn.linear_model import RidgeClassifier

etapes = ([
    ("std_scaler", StandardScaler()),
    ("smote", SMOTE()),
    ("clf_ridge", RidgeClassifier())
])

model_reg_ridge = Pipeline(steps=etapes)

In [ ]:
model_reg_ridge.fit(X_train, y_train)

In [ ]:
y_pred_reg_ridge = model_reg_ridge.predict(X_test)

#### B) Mesure de la performance

In [ ]:
score_reg_ridge = modele_reg_log.score(X_test, y_test)
print("Le score du modèle est : ", score_reg_ridge)

In [ ]:
# Calcul de la matrice de confusion
from sklearn.metrics import confusion_matrix

mat_conf_reg_ridge = confusion_matrix(y_test, y_pred_reg_ridge)
print("La matrice de confusion est : \n", mat_conf_reg_ridge)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

modalites =[0,1] 
fig, ax = plt.subplots()
tick_marks = np.arange(len(modalites))
plt.xticks(tick_marks, modalites)
plt.yticks(tick_marks, modalites)

sns.heatmap(pd.DataFrame(mat_conf_reg_ridge), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Matrice de confusion', y=1.1)
plt.ylabel('Valeur observée')
plt.xlabel('valeur prédite')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

y_pred_prob_reg_ridge = model_gradient_boosting.predict_proba(X_test)[:, 1] # probabilité que la classe soit 1
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_reg_ridge) # calcul des taux de faux positifs et vrais positifs
roc_auc = auc(fpr, tpr) # calcul de l'aire sous la courbe ROC

# Tracé de la courbe ROC
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--') # ligne en pointillés représentant la performance d'un classificateur aléatoire
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("Taux de faux positif (1 - spécificité)")
plt.ylabel('Taux de vrai positif (sensibilité)')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Calcul du score AUC

from sklearn.metrics import roc_auc_score

score_auc_reg_ridge = roc_auc_score(y_test, y_pred_prob_reg_ridge)
score_auc_reg_ridge

In [ ]:
# Calcul de la précision

from sklearn.metrics import precision_score

precision_reg_ridge = precision_score(y_test, y_pred_reg_ridge)
print("Precision:", precision_reg_ridge)

In [ ]:
from sklearn.metrics import recall_score

recall_reg_ridge = recall_score(y_test, y_pred_reg_ridge)
print("Recall:", recall_reg_ridge)

In [ ]:
from sklearn.metrics import f1_score

# Calculate and print F1-score
f1_reg_ridge = f1_score(y_test, y_pred_reg_ridge)
print("F1-Score:", f1_reg_ridge)

In [ ]:
# Résumé du modèle de gradient boosting

print("Modèle de gradient boosting \n")
print("Score AUC:", score_auc_reg_ridge) 
print("F1-Score:", f1_reg_ridge)

## I.3. Comparaison des modèles

In [ ]:
# Comparaison des modèles

print("Régression logistique \n")
print("Score AUC:", score_auc_reg_log)
print("F1-Score:", f1_reg_log)

print("Forêts aléatoires \n")
print("Score AUC:", score_auc_random_forest)
print("F1-Score:", f1_random_forest)

print("Gradient boosting \n")
print("Score AUC:", score_auc_gradient_boosting)
print("F1-Score:", f1_gradient_boosting)

print("Régression ridge \n")
print("Score AUC:", score_auc_reg_ridge)
print("F1-Score:", f1_reg_ridge)